# Traffic assignment (MSA)

Components:
1. Network
2. Network loading function 
3. Cost function
4. Assignment step
5. Check covergence.

Method covered:
1. MSA
2. FW with biseciton
3. FW with newton's method


Useful link:
https://github.com/bstabler/TransportationNetworks

In [1]:
import numpy as np
import pandas as pd
import time
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
### next check the SiouxFalls
instance_name = "SiouxFalls"

In [3]:
node_attr = f'network_data/{instance_name}/SiouxFalls_node.tntp'

node_attr = pd.read_csv(node_attr, skiprows=1, sep='\t', header=None, names=["Node","X","Y",";"])

node_attr = node_attr.drop(columns=[";"])

node_attr.head()

node_attr.to_csv(f'network_data/{instance_name}/network_node.csv', index=False)

In [4]:
filepath = f'network_data/{instance_name}/SiouxFalls_net.tntp'

In [5]:
net = pd.read_csv(filepath, skiprows=8, sep='\t')
# drop columns named "~", ";"
net = net.drop(columns=["~", ";"])

net

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,1,2,25900.200640,6,6,0.15,4,0,0,1
1,1,3,23403.473190,4,4,0.15,4,0,0,1
2,2,1,25900.200640,6,6,0.15,4,0,0,1
3,2,6,4958.180928,5,5,0.15,4,0,0,1
4,3,1,23403.473190,4,4,0.15,4,0,0,1
...,...,...,...,...,...,...,...,...,...,...
71,23,22,5000.000000,4,4,0.15,4,0,0,1
72,23,24,5078.508436,2,2,0.15,4,0,0,1
73,24,13,5091.256152,4,4,0.15,4,0,0,1
74,24,21,4885.357564,3,3,0.15,4,0,0,1


In [6]:
net["Edge"] = net.index
net1 = net[["Edge","init_node","term_node","free_flow_time","capacity","b","power"]]

In [7]:
# column names are: Edge,Node_F,Node_T,FFTT,Capacity,Beta,Alpha
net1.columns = ["Edge","Node_F","Node_T","FFTT","Capacity","Beta","Alpha"]

In [8]:
net1.to_csv(f'network_data/{instance_name}/network_net.csv', index=False)

In [9]:
od = pd.read_csv(f'network_data/{instance_name}/SiouxFalls_od.csv')
od.to_csv(f'network_data/{instance_name}/network_od0.csv', index=False)
od

,O,D,Ton
0,2,1,100.0
1,3,1,100.0
2,4,1,500.0
3,5,1,200.0
4,6,1,300.0
...,...,...,...
523,19,24,100.0
524,20,24,400.0
525,21,24,500.0
526,22,24,1100.0


In [11]:
so_sol = pd.read_csv(f'network_data/{instance_name}/SiouxFalls_flow.tntp', skiprows=1, sep='\t', header=None, names=["init_node","term_node","flow","cost"])

so_sol

,init_node,term_node,flow,cost
0,1,2,4494.657646,6.000816
1,1,3,8119.079948,4.008691
2,2,1,4519.079948,6.000834
3,2,6,5967.336396,6.573598
4,3,1,8094.657646,4.008587
...,...,...,...,...
71,23,22,9626.210200,12.243138
72,23,24,7902.983927,3.759304
73,24,13,11112.394731,17.617021
74,24,21,10259.524716,11.752579


In [13]:
fftt = net["free_flow_time"].to_numpy()
capacity = net["capacity"].to_numpy()
beta = net["b"].to_numpy()
alpha = net["power"].to_numpy()

In [19]:
# fftt*self.link_flow + capacity/(self.alpha+1)*self.fftt*self.beta*(self.link_flow/self.capacity)**(self.alpha+1)

so_sol1 = fftt*so_sol["flow"] + capacity/(alpha+1)*fftt*beta*(so_sol["flow"]/capacity)**(alpha+1)

In [20]:
so_sol1.sum()

4231335.28710744

In [23]:
ue_sol1 = fftt*ue["UE_flow"] + capacity/(alpha+1)*fftt*beta*(ue["UE_flow"]/capacity)**(alpha+1)

In [24]:
ue_sol1.sum()

4159408.538446357

In [ ]:
ue

In [ ]:
def create_network(file):
    net=pd.read_csv(file)
    G=nx.DiGraph()
    beta=[]
    alpha=[]
    capacity=[]
    fftt=[]
    for _,row in net.iterrows(): #iterate over all rows
        e,fn,tn,tt_,capacity_,beta_,alpha_=row #unpack your attributes
        G.add_edge(fn,tn,travel_time=tt_,travel_flow=0)
        beta.append(beta_);alpha.append(alpha_);fftt.append(tt_);capacity.append(capacity_)
    return G,beta,alpha,capacity,fftt

In [ ]:
#create graph
class TA:
    '''
    The class defines the rule for link-based traffic assignment
    '''
    def __init__(self,graph,od,od_demand,beta,alpha,tt,capacity,eps=1e-3):
        self.beta=beta
        self.alpha=alpha
        self.tt=tt
        self.capacity=capacity
        self.od=od
        self.G=graph
        self.nodes=G.nodes
        self.edges=G.edges
        self.od_demand=od_demand
        self.link_time=np.zeros(len(self.edges))
        self.link_flow=np.zeros(len(self.edges))
        self.eps=eps #stopping criteria
        self.iteration=0
    
    def update_travel_time(self):
        self.link_time=self.tt*(1+self.beta*(self.link_flow/self.capacity)**self.alpha)
        for e,tt in zip(self.G.edges,self.link_time):
            self.G[e[0]][e[1]]['travel_time']=tt #update link travel time in the graph
            
    def update_link_flow(self,link_flow,theta=1):
        TTST=sum(self.link_flow*self.link_time)
        SPTT=sum(link_flow*self.link_time)
        self.link_flow=(1-theta)*self.link_flow+theta*link_flow #convex combination
        for e,f in zip(self.G.edges,self.link_flow):
            self.G[e[0]][e[1]]['travel_flow']=f
        gap=(TTST/SPTT-1)
        return gap
    
    def compute_gradient(self,x1,x2,lbd):
        '''
        x1: the current link flow assignment
        x2: the shortest path flow assignment
        lbd: lambda ratio 
        '''
        flow=(1-lbd)*x1+lbd*x2 
        link_cost=(self.tt*(1+self.beta*(flow/self.capacity)**self.alpha))@(x2-x1)
        return link_cost
    
    def bisection(self,x1,x2):
        '''
        Use bisection to find the optimal lambda value
        x1: the current link flow assignment
        x2: the shortest path flow assignments
        we start with lbd=0 and 1 
        '''
        l=0;l_=1
        n=0 #iteration
        while abs(l_-l)>1e-1: #stopping criteria
#             c1=self.compute_gradient(x1,x2,l)
#             c2=self.compute_gradient(x1,x2,l_)
            c=self.compute_gradient(x1,x2,(l+l_)/0.5)
            if c>0:
                l_=l+(l_-l)/2
            else:
                l=l+(l_-l)/2
            n+=1
        return (l+l_)/2,n
                
        
    def step_FW(self):
        if self.iteration==0:
            self.link_flow=self.find_sp_flow()
        
        t1=time.time()
        current_flow=self.link_flow
        self.update_travel_time()
        new_flow=self.find_sp_flow()
        lbd,n=self.bisection(current_flow,new_flow)
        print('Iteration={},lambda={},bisection iterations={}'.format(self.iteration,lbd,n))
        gap=self.update_link_flow(new_flow,lbd)
        cost=time.time()-t1
        self.iteration+=1
        return gap

    def find_sp_flow(self):
        sp=self.get_sp() #list of the shortest path for all od pairs
        myflow=dict.fromkeys(self.edges,0)
        for i in range(len(sp)): #go through all od pairs
            for j in range(len(sp[i])-1): #go through all the links on each sp
                    myflow[(sp[i][j],sp[i][j+1])]+=self.od_demand[i]
        sp_link_flow=np.array(list(myflow.values()))
        return sp_link_flow
    
    def step_MSA(self):
        if self.iteration==0:
            self.link_flow=self.find_sp_flow()
        t1=time.time()
        new_link_flow=self.find_sp_flow()
        cost=time.time()-t1
        t1=time.time()
        gap=self.update_link_flow(new_link_flow,1/(self.iteration+1))
        cost=time.time()-t1
        self.iteration+=1
        return gap
    

    def get_sp(self): #return shortest path of all od pairs
        return [nx.dijkstra_path(self.G,self.od[i][0],self.od[i][1],weight='travel_time') for i in range(len(self.od))]
    
    def get_total_cost(self):
        return np.sum(self.link_time*self.link_flow)

In [ ]:
G,beta,alpha,capacity,fftt=create_network('network_data/network.csv')
print(len(G.edges))
print(len(beta),len(alpha),len(capacity),len(fftt))

pos=nx.spectral_layout(G) 
nx.draw(G,pos,with_labels=True,node_size=100,node_color='r',font_size=20,arrowsize=20) #visualize layout

In [ ]:
ods=[['N002','N003'],['N005','N009']]
ods_demand=[1000,2000]
model=TA(G,ods,ods_demand,beta,alpha,fftt,capacity)

gaps_MSA=[]
iterations=[]
for it in range(100):
    gap=model.step_MSA()
    if it%10==0:
        gaps_MSA.append(gap)
        iterations.append(model.iteration)
        
ods=[['N002','N003'],['N005','N009']]
ods_demand=[1000,2000]
model=TA(G,ods,ods_demand,beta,alpha,fftt,capacity)
gaps_FW=[]
iterations=[]
for it in range(100):
    gap=model.step_FW()
    if it%10==0:
        gaps_FW.append(gap)
        iterations.append(model.iteration)

In [ ]:
fig,ax=plt.subplots(figsize=(6,4))
ax.plot(iterations,gaps_FW,label='FW')
ax.plot(iterations,gaps_MSA,label='MSA')
ax.set_yscale('log')
ax.set_xlabel('Iteration')
ax.set_ylabel('Relative Gap')
ax.legend()

In [ ]:
print(model.link_flow)
fig,ax=plt.subplots(figsize=(15,15))
pos=nx.spectral_layout(model.G) 
nx.draw(model.G,pos,with_labels=True)

node_labels = nx.get_node_attributes(model.G,'name')
nx.draw_networkx_labels(G, pos, labels = node_labels)
edge_labels = nx.get_edge_attributes(G,'travel_flow')
nx.draw_networkx_edge_labels(G, pos, edge_labels)

In [ ]:
#bisection  f=1+3x
def fun(x1,x2,lbd):
    x=(1-lbd)*x1+lbd*x2
    z=(1+3*x)@(x2-x1)
    return z
lbd=0.3
x1=np.array([2,3])
x2=np.array([3,2])
fun(x1,x2,lbd)

In [ ]:
def bisect(x1,x2,a,b,eps=1e-3):
    if abs(a-b)<1e-3:
        return abs(b-a)/2
    else:
        print(a,b)
        f1=fun(x1,x2,a)
        f2=fun(x1,x2,b)
        c=a+(b-a)/2
        print(c)
        f3=fun(x1,x2,c)
        if f3>0:
            a=c
        else:
            b=c
        return bisect(x1,x2,a,b)

In [ ]:
lbda=bisect(x1,x2,0,1)